In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#Get all links from 2025 onwards every 6 hours - gdelt_latest_export_links_per_day_2025.txt

# import re
# import requests
# from datetime import datetime, timedelta

# master_url = "http://data.gdeltproject.org/gdeltv2/masterfilelist.txt"
# pattern = re.compile(r'http://data\.gdeltproject\.org/gdeltv2/(\d{14})\.export\.CSV\.zip')
# start_date = datetime(2025, 1, 1)
# output_file = "gdelt_latest_export_links_per_6h_2025.txt"

# response = requests.get(master_url)
# latest_per_slot = {}

# for line in response.text.splitlines():
#     match = pattern.search(line)
#     if match:
#         file_datetime = datetime.strptime(match.group(1), '%Y%m%d%H%M%S')
#         if file_datetime >= start_date:
#             # Calculate the 6-hour slot for this timestamp
#             slot_hour = (file_datetime.hour // 6) * 6
#             slot_start = file_datetime.replace(hour=slot_hour, minute=0, second=0)
#             slot_key = slot_start.strftime('%Y%m%d%H')
#             # If this is the first file for the slot or later than the current, keep it
#             if (slot_key not in latest_per_slot) or (file_datetime > latest_per_slot[slot_key][0]):
#                 latest_per_slot[slot_key] = (file_datetime, match.group(0))

# # Sort by slot and write URLs to output file
# with open(output_file, "w") as f:
#     for slot in sorted(latest_per_slot):
#         f.write(latest_per_slot[slot][1] + "\n")

# print(f"Wrote {len(latest_per_slot)} URLs (one per 6-hour slot) to {output_file}")


In [ ]:
#Get all links from 2025 onwards every 15 min - gdelt_latest_export_links_per_day_2025.txt

import re
import requests
from datetime import datetime, timedelta

master_url = "http://data.gdeltproject.org/gdeltv2/masterfilelist.txt"
pattern = re.compile(r'http://data\.gdeltproject\.org/gdeltv2/(\d{14})\.export\.CSV\.zip')
start_date = datetime(2025, 1, 1)
output_file = "gdelt_latest_export_links_per_15min_2025.txt"

response = requests.get(master_url)
latest_per_slot = {}

for line in response.text.splitlines():
    match = pattern.search(line)
    if match:
        file_datetime = datetime.strptime(match.group(1), '%Y%m%d%H%M%S')
        if file_datetime >= start_date:
            # Calculate the 15-minute slot for this timestamp
            minute_slot = (file_datetime.minute // 15) * 15
            slot_start = file_datetime.replace(minute=minute_slot, second=0)
            slot_key = slot_start.strftime('%Y%m%d%H%M')
            # Keep the latest file for each slot
            if (slot_key not in latest_per_slot) or (file_datetime > latest_per_slot[slot_key][0]):
                latest_per_slot[slot_key] = (file_datetime, match.group(0))

# Sort by slot and write URLs to output file
with open(output_file, "w") as f:
    for slot in sorted(latest_per_slot):
        f.write(latest_per_slot[slot][1] + "\n")

print(f"Wrote {len(latest_per_slot)} URLs (one per 15-minute slot) to {output_file}")


In [ ]:
#Get all topics related to the US tariffs - us_tariff_results.csv

import requests
import zipfile
import io
import csv
import re

url_file = 'C://Users//Petarche//Downloads/gdelt_latest_export_links_per_15min_2025.txt'  # Your file with the list of URLs
output_file = 'us_tariff_results.csv'

# Keywords
keywords = [
    'trade deal', 'us tariff', 'u.s. tariff', 'tariff', 'tariffs', 'US tariffs' 'united states tariff', 'american tariff',
    'trump tariff', 'biden tariff', 'trade war', 'import duty', 'steel tariff', 'china tariff'
]
keyword_pattern = re.compile('|'.join(keywords), re.IGNORECASE)

with open(url_file, 'r') as f:
    urls = [line.strip() for line in f if line.strip()]

with open(output_file, 'w', newline='', encoding='utf-8') as outcsv:
    writer = csv.writer(outcsv)
    writer.writerow(['SourceFile', 'GDELT_Row'])  # Header

    for url in urls:
        print(f"Processing: {url}")
        try:
            r = requests.get(url, timeout=60)
            z = zipfile.ZipFile(io.BytesIO(r.content))
            # Find the CSV file inside the zip
            csvname = [name for name in z.namelist() if name.endswith('.CSV')][0]
            with z.open(csvname) as csvfile:
                for row in csvfile:
                    try:
                        # Decode and search for keywords in the row
                        row_str = row.decode('utf-8', errors='ignore')
                        if keyword_pattern.search(row_str):
                            writer.writerow([url, row_str.strip()])
                    except Exception as e:
                        continue  # skip problematic lines
        except Exception as e:
            print(f"Failed to process {url}: {e}")

print(f"Done! Results saved in {output_file}")

In [ ]:
!pip install trafilatura

In [ ]:
#Extract the other data from the GDELT_Row from us_tariff_results.csvas separate columns - merged_columns.csv

import csv


GDELT_COLUMNS = [
    "event_id", "event_date", "event_month", "event_year", "event_year_decimal",
    "actor1_country_code", "actor1_country_name", "actor1_code", "actor1_type",
    "actor1_code_num", "event_code", "event_root_code", "quad_class", "goldstein_scale",
    "num_mentions", "num_sources", "num_articles", "avg_tone", "actor1_geo_type",
    "actor1_geo_fullname", "actor1_geo_country", "actor1_geo_adm1", "actor1_geo_adm2",
    "actor1_geo_lat", "actor1_geo_long", "actor1_geo_feature_id", "actor2_geo_type",
    "actor2_geo_type_dup", "actor2_geo_fullname", "actor2_geo_country", "actor2_geo_adm1",
    "actor2_geo_adm2", "actor2_geo_lat", "actor2_geo_long", "actor2_geo_feature_id",
    "event_timestamp", "source_url"
]

input_csv = 'us_tariff_results.csv'
output_csv = 'merged_columns.csv'

def parse_gdelt_row(row_str):
    # Split on spaces
    parts = row_str.strip().split()
    if len(parts) < 37:
        # Not enough fields, skip
        return None
    # The last field is always the URL
    url = parts[-1]
    fields = parts[:-1]
    # If there are more than 36 fields, merge extras into the last field before URL (for city names with spaces)
    if len(fields) > 36:
        fields = fields[:35] + [' '.join(fields[35:])]
    return fields + [url]

with open(input_csv, newline='', encoding='utf-8') as infile, \
     open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile, fieldnames=['SourceFile', 'GDELT_Row'])
    writer = csv.writer(outfile)
    # Write header
    writer.writerow(['SourceFile'] + GDELT_COLUMNS)
    for row in reader:
        sourcefile = row['SourceFile']
        gdelt_row = row['GDELT_Row']
        fields = parse_gdelt_row(gdelt_row)
        if not fields:
            continue
        writer.writerow([sourcefile] + fields)

print(f"Done! Data saved in {output_csv}")


In [ ]:
#Trafilatura extracts all the news articles in a separate column from the links from merged_columns.csv - extracted_articles.csv

import csv
import trafilatura

input_csv = 'C://Users//Petarche//Downloads/merged_columns.csv'
output_csv = 'extracted_texts_all.csv'
max_articles = 1000000
skip_domains = ['cbc.ca', 'timescolonist.com', 'malaya.com']
seen_links = set()

def get_article_text(url):
    try:
        downloaded = trafilatura.fetch_url(url)
        if downloaded:
            text = trafilatura.extract(downloaded)
            return text if text else ""
        else:
            return ""
    except Exception as e:
        print(f"Failed to fetch {url}: {e}")
        return ""

with open(input_csv, newline='', encoding='utf-8') as infile, \
     open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.reader(infile)
    header = next(reader)

    # Find required column indices
    try:
        url_col_idx = header.index('source_url')
    except ValueError:
        raise Exception("No 'source_url' column found in the input file.")
    try:
        event_date_col_idx = header.index('event_date')
    except ValueError:
        raise Exception("No 'event_date' column found in the input file.")

    # All other columns except event_date and source_url
    other_columns = [col for i, col in enumerate(header) if i not in (event_date_col_idx, url_col_idx)]

    # New header order
    new_header = ['id', 'event_date', 'source_url', 'text', 'trust_factor'] + other_columns
    writer = csv.writer(outfile)
    writer.writerow(new_header)

    article_count = 0
    for row in reader:
        if article_count >= max_articles:
            break
        link = row[url_col_idx]
        if not (link.startswith("http://") or link.startswith("https://")):
            continue
        if any(domain in link for domain in skip_domains):
            continue
        if link in seen_links:
            continue
        seen_links.add(link)
        event_date = row[event_date_col_idx]
        other_values = [val for i, val in enumerate(row) if i not in (event_date_col_idx, url_col_idx)]
        print(f"[{article_count+1}] Processing: {link}")
        text = get_article_text(link)
        new_row = [article_count+1, event_date, link, text, 1] + other_values
        writer.writerow(new_row)
        article_count += 1

print(f"Done! Extracted {article_count} articles. Data saved in {output_csv}")


In [ ]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("extracted_texts_all.csv")
df

In [ ]:
df.info()

In [ ]:
df.dropna(inplace=True)


In [ ]:
df.info()

In [ ]:
# We think that event_month, event_year and event_year_decimal* should be dropped just because they are redundant values and we have that data in one column "event_date"
# also the column "id" because is just a sequential number with no meaningful information.



In [ ]:
df.drop(columns=['id'], inplace=True)
df

In [ ]:
df.to_csv('cleaned_nan_v1.csv', index=False)


In [ ]:
num_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")


In [ ]:
redundant_cols = [col for col in df.columns if df[col].nunique() == 1]
print(f"Redundant columns: {redundant_cols}")

# The output is trust_factor , because right now all rows with that column
# are with values = 1 , which is made by purpose from us , until we fine tune a model, so it can choose the trust factor


In [ ]:
# This five ['goldstein_scale', 'num_mentions', 'num_sources', 'num_articles', 'avg_tone'] columns are of type object,
# most of them have numerical value, bit still it contains some text, like WORLD,BANK,MNC etc...
#

#numeric_cols = ['goldstein_scale', 'num_mentions', 'num_sources', 'num_articles', 'avg_tone']
#df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

#
df["num_articles"].value_counts()

In [ ]:
numeric_df = df.select_dtypes(include=['int64', 'float64'])
plt.figure(figsize=(12, 8))
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Matrix for Numeric Columns")
plt.show()


In [ ]:
# As we can see from the correlation heatmap the following columns are pretty highly correlated, >0.7  , they should be dropped (["event_year_decimal","event_month", "event_year"])

In [ ]:
df = df.drop(columns=['event_year_decimal','event_month', 'event_year'])

print(df.info())


In [ ]:
df.actor1_geo_country.value_counts()

In [ ]:
df.actor1_country_code.value_counts()

In [ ]:
# TO DISCUSS WITH PROFESSOR:

#We think the following columns are likely redundant pairs

#actor1_country_code & actor1_country_name (we think we should keep one)

#actor2_geo_type & actor2_geo_type_dup (we need to see if they are duplicates then we should drop duplicates)

#actor1_geo_country & actor1_country_code (geographic redundancy maybe?)

#SourceFile (file paths rarely add value)

In [ ]:
# List of columns for each actor
actor1_geo_cols = [
    'actor1_geo_type', 'actor1_geo_fullname', 'actor1_geo_country',
    'actor1_geo_adm1', 'actor1_geo_adm2'
]
actor2_geo_cols = [
    'actor2_geo_type', 'actor2_geo_fullname', 'actor2_geo_country',
    'actor2_geo_adm1', 'actor2_geo_adm2'
]

# Summary statistics
print(df[actor1_geo_cols].describe(include='all'))
print(df[actor2_geo_cols].describe(include='all'))

# Top 10 most frequent values for each column
for col in actor1_geo_cols + actor2_geo_cols:
    print(f"\nTop 10 values for {col}:")
    print(df[col].value_counts().head(10))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for col in actor1_geo_cols + actor2_geo_cols:
    plt.figure(figsize=(10,4))
    sns.countplot(data=df, y=col, order=df[col].value_counts().head(10).index)
    plt.title(f"Top 10 {col} values")
    plt.tight_layout()
    plt.show()


In [ ]:
# TO DISCUSS WITH PROFESSOR:
# Some rows are int and some other rows have strings, because of that we can't come to any conclusions with the data we currently have, which is a bit mixed.